# Imports and Functions 

In [ ]:
import os
import sys
import tensorflow.compat.v1 as tf

import tensorflow_hub as hub
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import csv
from PIL import Image, ImageFile
import tensorflow as tf
#import tensorflow_datasets as tfds
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.utils.layer_utils import count_params

from keras.preprocessing.image import img_to_array
import math
from mpl_toolkits.axes_grid1 import ImageGrid
Image.MAX_IMAGE_PIXELS = 1000000000
ImageFile.LOAD_TRUNCATED_IMAGES = True

def load_wikidata(train_path, val_path, test_path, batch_size = 4, image_size = 223):
    print("Loading data...")
    print()
    datagen_kwargs = dict(rescale=1./255)
    dataflow_kwargs = dict(target_size=(image_size, image_size),batch_size=batch_size, interpolation="bilinear")
    
    valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
    valid_generator = valid_datagen.flow_from_directory(val_path, shuffle=False, **dataflow_kwargs)
    
    train_datagen = valid_datagen
    train_generator = train_datagen.flow_from_directory(train_path, shuffle=True, **dataflow_kwargs)
    
    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_directory(test_path, shuffle=False, **dataflow_kwargs)
    
    label_map = (train_generator.class_indices)
    print("Label map:")
    print(label_map)
    
    return train_generator, valid_generator, test_generator

#func to get effnet model
def unfreeze_effnet(model, num_unfreeze):
    for layer in model.layers[:-num_unfreeze]:
        layer.trainable = False
        
    print("Unfroze {} layers".format(num_unfreeze))
    return model
    
#train model
def train_effnetv2(model, train_generator, valid_generator, num_epochs, learning_rate, decay = False,
                   restore = False,\
                   checkpoint_path = "/projectnb/dl523/projects/Sarcasm/cp.ckpt",\
                   model_path = None, momentum = 0.9):
    
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    checkpoint_dir = os.path.dirname(checkpoint_path)
    
    # Save checkpoints in case training session is cut off
    callback_list = []
    callback_list.append(tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, verbose=1))
    callback_list.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1))
    
    if not restore:
        model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum), 
          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0.1),
          metrics=['accuracy', 'top_k_categorical_accuracy'])
    
    steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = valid_generator.samples // valid_generator.batch_size
    
    hist = model.fit(train_generator, epochs = num_epochs,\
                     validation_data = valid_generator,\
                     steps_per_epoch = steps_per_epoch,\
                     validation_steps = validation_steps, \
                     callbacks = callback_list).history
    
    
    
    #save model as a whole to share with the others
    if model_path != None:
        model.save(model_path)
    
    return model, hist

def eff_transform(img):
    eff_tensor = img.resize((223, 223))
    eff_tensor = img_to_array(eff_tensor)
    eff_tensor = eff_tensor*(1/255)
    eff_tensor = eff_tensor.reshape((1,eff_tensor.shape[0],eff_tensor.shape[1],eff_tensor.shape[2]))
    return eff_tensor

# For dynamically adjusting the size of the image grid
def largest_factor_pair(dim):
    factor_pairs = []
    for i in range(1, int(math.sqrt(dim))+1):
        if dim % i == 0:
            factor_pairs.append((i, dim / i))
    return factor_pairs[-1]


In [ ]:
# Enable GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install tensorflow_addons

import os
import sys
import tensorflow.compat.v1 as tf

# Download source code.
if "efficientnetv2" not in os.getcwd():
    !git clone --depth 1 https://github.com/google/automl
    os.chdir('automl/efficientnetv2')
    sys.path.append('.')
else:
    !git pull

def download(m):
    if m not in os.listdir():
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{m}.tgz
        !tar zxf {m}.tgz
    ckpt_path = os.path.join(os.getcwd(), m)
    return ckpt_path

import effnetv2_model

In [ ]:
Image.MAX_IMAGE_PIXELS = 1000000000
data_dir_train = '/projectnb/dl523/projects/Sarcasm/content/gdrive/Shareddrives/520 Project/Data/wikipaintings_full_aug/train'
data_dir_val = '/projectnb/dl523/projects/Sarcasm/wikipaintings_full/wikipaintings_val'
data_dir_test = '/projectnb/dl523/projects/Sarcasm/wikipaintings_full/wikipaintings_test'

# Locations to save model info
checkpoint_path = "/projectnb/dl523/projects/Sarcasm/cp.ckpt"
model_path = '/projectnb/dl523/projects/Sarcasm/effnetv2_model2'

IMAGE_SIZE = 223
BATCH_SIZE =  32
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

# Train or Load Model

In [ ]:
# https://github.com/google/automl/issues/1111
import effnetv2_model
x = tf.keras.Input(shape=[223, 223, 3])
effnet = effnetv2_model.get_model('efficientnetv2-l', include_top=False, weights='imagenet21k')
y = effnet.call(x)
y = tf.keras.layers.Dropout(rate=0.2)(y)
y = tf.keras.layers.Dense(25, activation='softmax')(y)
effnet_test = tf.keras.Model(inputs=[x], outputs=y)

In [ ]:
# Data loaders
train,val,test = load_wikidata(data_dir_train,data_dir_val,data_dir_test,batch_size = 32)
# Freezing layers 
effnet_test = unfreeze_effnet(effnet_test,36)

In [ ]:
# Training the model, or loading the pretrained model

if os.path.isdir(model_path):
    # If already trained, load and evaluate
    print('Loading Model...')
    effnet_features = tf.keras.models.load_model(model_path)
else:
    effnet_features,hist = train_effnetv2(effnet_test,train,val,4,.001,checkpoint_path = checkpoint_path, model_path = model_path)


# Feature Visualizations

In [ ]:
features_list = [layer.output for layer in effnet_features.layers]

feature_map_extractor = tf.keras.Model(
    inputs=effnet_features.input,
    outputs=features_list
)
TEST_IMAGE = '/projectnb2/dl523/students/colejh/520/0x0.jpg'
img = Image.open(TEST_IMAGE)
eff_tensor = eff_transform(img)
feature_map = feature_map_extractor.predict(eff_tensor)

In [ ]:
# Last layer for each size of filter
desired_layers = [5,12,19,29,48,73,79] 
selected = [feature_map[i] for i in desired_layers]

# Graph each of the feature maps from selected layers
for index,images in enumerate(selected):
    size = images.shape[3]
    largest_dimensions = largest_factor_pair(size)
    height = int(largest_dimensions[0])
    width = int(largest_dimensions[1])

    fig = plt.figure(figsize=(35., 35.))
    grid = ImageGrid(fig, 111,  
                     nrows_ncols=(height, width), 
                     axes_pad=0.3, 
                     )
    img_list = []
    for idx in range(size):
        img_list.append(images[0][:,:,idx])
    
    for ax, im in zip(grid, img_list):
        ax.imshow(im,cmap = 'gray')
    
    fig.suptitle('Feature Maps for Layer {name}\'s {filt} filters'.format(name = effnet_features.layers[desired_layers[index]].name,filt = size),y=1.05, fontsize=18)
    plt.show()
    